In [1]:
import pandas as pd
import numpy as np

In [2]:
columns = [f'V{i}' for i in range(1, 29)]

# add more col name to columns
columns.insert(0, 'time')
columns.append('amount')
columns.append('target')
columns

# read data
df = pd.read_csv('./data/card.csv', names=columns)


In [3]:
# view data
df.head()

,time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,amount,target
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [4]:
# get the predictor and traget variables

X = df.loc[:, 'time':'amount']
y = df['target']

In [5]:
# shape pf the data
X.shape, y.shape

((284807, 30), (284807,))

In [6]:
# test data into train_validate and test
from sklearn.model_selection import train_test_split

X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2, random_state=33)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=(1/8), random_state=33)

In [7]:
# shape of the data
X_train.shape, X_val.shape, X_test.shape, y_train.shape, y_val.shape, y_test.shape


((199364, 30), (28481, 30), (56962, 30), (199364,), (28481,), (56962,))

In [36]:
# scale the data
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

In [53]:


hyper_param_options = [
    {'hidden_layer_sizes': (30), 'activation': 'relu', 'alpha': 0.0001, 'batch_size': 100, 'learning_rate_init': 0.001,   'random_state': 33,  'verbose': True},
    {'hidden_layer_sizes': (30,20,10,5,2), 'activation': 'relu', 'alpha': 0.0001, 'batch_size': 100, 'learning_rate_init': 0.001,   'random_state': 33,  'verbose': True},
]

def train_model(id, hyper_params):
    mlp_classifier = MLPClassifier(**hyper_params).fit(X_train_scaled, y_train)
    train_score = mlp_classifier.score(X_train_scaled, y_train)
    val_score = mlp_classifier.score(X_val_scaled, y_val)
    test_score = mlp_classifier.score(X_test_scaled, y_test)
    return {
        'id': id,
        'hidden_layers': mlp_classifier.n_layers_ - 2,
        'architecture': mlp_classifier.hidden_layer_sizes,
        'activation': hyper_params['activation'],
        'alpha': hyper_params['alpha'],
        'batch_size': hyper_params['batch_size'],
        'learning_rate': hyper_params['learning_rate_init'],
        'iterations': mlp_classifier.n_iter_,
        'train_score': train_score,
        'val_score': val_score,
        'test_score': test_score,
    }


In [54]:
observations_df = pd.DataFrame()

for index,hyper_param_option in enumerate(hyper_param_options):
    print(f"Training model {index}, with hyper params: {hyper_param_option}")
    result = train_model(index, hyper_param_option)
    result_df = pd.DataFrame([result])
    observations_df = pd.concat([observations_df, result_df], ignore_index=True)  
    print(f"Training model {index}, with hyper params: {hyper_param_option} completed")
    print("Test results: ", observations_df.iloc[index])

observations_df.to_csv('./data/observations.csv', index=False)

Training model 0, with hyper params: {'hidden_layer_sizes': 30, 'activation': 'relu', 'alpha': 0.0001, 'batch_size': 100, 'learning_rate_init': 0.001, 'random_state': 33, 'verbose': True}
Iteration 1, loss = 0.02605554
Iteration 2, loss = 0.00352391
Iteration 3, loss = 0.00313689
Iteration 4, loss = 0.00281437
Iteration 5, loss = 0.00266319
Iteration 6, loss = 0.00251309
Iteration 7, loss = 0.00237758
Iteration 8, loss = 0.00223661
Iteration 9, loss = 0.00218227
Iteration 10, loss = 0.00212099
Iteration 11, loss = 0.00206419
Iteration 12, loss = 0.00192984
Iteration 13, loss = 0.00193737
Iteration 14, loss = 0.00189214
Iteration 15, loss = 0.00183911
Iteration 16, loss = 0.00175570
Iteration 17, loss = 0.00173121
Iteration 18, loss = 0.00172236
Iteration 19, loss = 0.00169658
Iteration 20, loss = 0.00169747
Iteration 21, loss = 0.00167529
Iteration 22, loss = 0.00158047
Iteration 23, loss = 0.00157947
Training loss did not improve more than tol=0.000100 for 10 consecutive epochs. Stopp

In [55]:
observations_df

,id,hidden_layers,architecture,activation,alpha,batch_size,learning_rate,iterations,train_score,val_score,test_score
0,0,1,30,relu,0.0001,100,0.001,23,0.999679,0.999157,0.999368
1,1,5,"(30, 20, 10, 5, 2)",relu,0.0001,100,0.001,35,0.999839,0.999333,0.999350


In [49]:
(mlp_classifier.hidden_layer_sizes)

(100,)

In [43]:
np.logspace(-1, 1, 5)


array([ 0.1       ,  0.31622777,  1.        ,  3.16227766, 10.        ])